In [1]:
from architype.architype.dataset.build import ArchiMateDataset

dataset_dir = "architype/data/raw/eamodelset"
archimate_dataset = ArchiMateDataset(
    dataset_dir,
    language="en",
    min_edges=5,
    min_enr=1.2,
    dummy_ratio_threshold=0.5,
    duplicate_overlap_threshold=0.95,
    # reload=True
)

/Users/sali/VSCodeRepos/archi-type-prediction/.venv/lib/python3.13/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/Context.cpp:85.)
  _C._set_float32_matmul_precision(precision)


Loading archimate from pickle
Loaded archimate with 295 graphs
Loaded archimate with 278 graphs
Graphs: 278


In [2]:
archimate_dataset[0].nodes(data='type')

NodeDataView({'3d74c0be': 'ApplicationProcess', '39e16f4f': 'ApplicationProcess', 'eddc9a3a': 'ApplicationComponent', '4e17cc0f': 'ApplicationFunction', 'f37e5d1c': 'ApplicationFunction', '4d9a4134': 'ApplicationProcess', 'af0ce369': 'ApplicationProcess', 'c0482ce5': 'ApplicationProcess', '9912acc6': 'ApplicationComponent', '313fd5ef': 'ApplicationFunction', 'f7864475': 'ApplicationProcess', '41032782': 'ApplicationFunction', 'fc182f7f': 'ApplicationFunction', '8683eaec': 'ApplicationFunction', '5245d9ca': 'ApplicationComponent', 'a840b94a': 'ApplicationInterface', 'f5277413': 'ApplicationFunction', 'a00d26b2': 'ApplicationFunction', '8df90b12': 'ApplicationFunction', '2739bdf6': 'ApplicationComponent', '6cbab361': 'ApplicationFunction', 'aff5c846': 'ApplicationProcess', '411e9964': 'ApplicationFunction', '5ec4146f': 'ApplicationFunction', '5c92c2f4': 'ApplicationFunction', 'c3a9d7ce': 'ApplicationComponent', '265f513d': 'ApplicationInterface', 'dcc3afe9': 'ApplicationProcess', '6ce034

In [3]:
len(archimate_dataset)

278

In [4]:
graphs, flagged = archimate_dataset.llm_filter_graphs(threshold=0.5)

Scoring graphs:   0%|          | 0/278 [00:00<?, ?graph/s]

Loading response from /Users/sali/VSCodeRepos/archi-type-prediction/.tmp/prompts/model_meaningfulness_batch_1_response.json
Loading response from /Users/sali/VSCodeRepos/archi-type-prediction/.tmp/prompts/model_meaningfulness_batch_2_response.json
Loading response from /Users/sali/VSCodeRepos/archi-type-prediction/.tmp/prompts/model_meaningfulness_batch_1_response.json
Loading response from /Users/sali/VSCodeRepos/archi-type-prediction/.tmp/prompts/model_meaningfulness_batch_2_response.json
Loading response from /Users/sali/VSCodeRepos/archi-type-prediction/.tmp/prompts/model_meaningfulness_batch_3_response.json
Loading response from /Users/sali/VSCodeRepos/archi-type-prediction/.tmp/prompts/model_meaningfulness_batch_4_response.json
Loading response from /Users/sali/VSCodeRepos/archi-type-prediction/.tmp/prompts/model_meaningfulness_batch_1_response.json
Loading response from /Users/sali/VSCodeRepos/archi-type-prediction/.tmp/prompts/model_meaningfulness_batch_1_response.json
Loading 

In [5]:
len(flagged), len(graphs), len(archimate_dataset)

(33, 245, 278)

In [6]:
node_texts = archimate_dataset.get_node_texts(
    node_cls_label="layer",
    test_size=0.2,
    distance=1,
    use_node_types=True
)

Getting node texts:   0%|          | 0/278 [00:00<?, ?it/s]

In [7]:
node_texts['train']

[{'text': 'get iterator from aggregate() <> next item in iterator(ApplicationProcess, application)',
  'label': 'application',
  'graph_id': '47233919',
  'node_id': '3d74c0be'},
 {'text': 'next item in iterator() <> if not end(ApplicationFunction, application)',
  'label': 'application',
  'graph_id': '47233919',
  'node_id': '39e16f4f'},
 {'text': 'creator() <> create iterator(ApplicationFunction, application)\ncreator() <> construct algoritm()',
  'label': 'application',
  'graph_id': '47233919',
  'node_id': 'eddc9a3a'},
 {'text': 'create iterator() <> call fabric(ApplicationProcess, application)\ncreate iterator() <> return iterator(ApplicationProcess, application)\ncreate iterator() <> create iterator A (this) (copy)(ApplicationProcess, application)',
  'label': 'application',
  'graph_id': '47233919',
  'node_id': '4e17cc0f'},
 {'text': 'step()',
  'label': 'application',
  'graph_id': '47233919',
  'node_id': 'af0ce369'},
 {'text': 'call fabric() <> step()',
  'label': 'applica

In [9]:
edge_texts = archimate_dataset.get_edge_texts(
    edge_cls_attribute="type",
    test_size=0.2,
    distance=1,
    use_edge_types=True,
    use_node_types=True
)

Getting edge texts:   0%|          | 0/278 [00:00<?, ?it/s]

In [11]:
edge_texts['train'][0]

{'text': 'get iterator from aggregate(ApplicationProcess, application)<edge_begin><edge_end>next item in iterator(ApplicationProcess, application) <> if not end(ApplicationFunction, application)',
 'label': 'Flow',
 'graph_id': '47233919',
 'edge': ('3d74c0be', '39e16f4f')}

In [ ]:
len(node_texts['train']), len(edge_texts['train'])

(24148, 36910)

In [ ]:
len(node_texts['test']), len(edge_texts['test'])

(5887, 9087)

Okay now that the dataset generation is working such that `archimate_dataset.get_edge_texts` and `archimate_dataset.get_node_texts` provides a list of train and test data - 

Dict[str, List[Dict[str, Any]]]
List of samples like this - 
{'text': 'get iterator from aggregate(ApplicationProcess, application)<edge_begin><edge_end>next item in iterator(ApplicationProcess, application) <> if not end(ApplicationFunction, application)',
 'label': 'Flow',
 'graph_id': '47233919',
 'edge': ('3d74c0be', '39e16f4f')}

Given this type of data, next you need to implement the models package architype. 
The package needs to have a base.py which is the abstract class that exposes a general api for all the three approaches - api, bert and unsloth.

The bert approach needs to support the following - 
- takes the name of a huggingface bert model and a ModelsDataset and finetunes the model for text classification
- takes the name of a model which can be an huggingface model or a finetuned model and runs inference on the dataset